In [1]:
import os
import time
import datetime
from shutil import copyfile
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import xarray as xr
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import json
import copy
from torchvision.utils import make_grid
from torch.utils.data import Dataset
from torchvision import transforms
from torchvision.transforms import v2
import torchvision.models as models
from torch.utils.data import DataLoader
import utils

In [2]:
BASE_PATH_DATA = 'data/skogsstyrelsen/'
BAND_NAMES = ['b01', 'b02', 'b03', 'b04', 'b05', 'b06', 'b07', 'b08', 'b8a', 'b09', 'b11', 'b12']
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
# Read data + corresponding json info (incl ground truth)
img_paths_train = list(np.load(os.path.join(BASE_PATH_DATA, 'skogs_names_train.npy')))
img_paths_train = [path[1:] for path in img_paths_train]

img_paths_val = list(np.load(os.path.join(BASE_PATH_DATA, 'skogs_names_val.npy')))
img_paths_val = [path[1:] for path in img_paths_val]

img_paths_test = list(np.load(os.path.join(BASE_PATH_DATA, 'skogs_names_test.npy')))
img_paths_test = [path[1:] for path in img_paths_test]

json_content_train = list(np.load(os.path.join(BASE_PATH_DATA, 'skogs_json_train.npy'), allow_pickle=True))
json_content_val = list(np.load(os.path.join(BASE_PATH_DATA, 'skogs_json_val.npy'), allow_pickle=True))
json_content_test = list(np.load(os.path.join(BASE_PATH_DATA, 'skogs_json_test.npy'), allow_pickle=True))

train_label = list(np.load(os.path.join(BASE_PATH_DATA, "skogs_gts_train.npy")))
val_label = list(np.load(os.path.join(BASE_PATH_DATA, "skogs_gts_val.npy")))
test_label = list(np.load(os.path.join(BASE_PATH_DATA, "skogs_gts_test.npy")))

In [4]:
def load_image(path):
    img = xr.open_dataset(path)
    yy_mm_dd = getattr(img, 'time').values[0]
    yy = yy_mm_dd.astype('datetime64[Y]').astype(int) + 1970
    mm = yy_mm_dd.astype('datetime64[M]').astype(int) % 12 + 1

    band_list = []
    for band in BAND_NAMES:
        if yy >= 2022 and mm >= 1: # New normalization after Jan 2022
            band_list.append((getattr(img, band).values - 1000) / 10000)
        else:
            band_list.append(getattr(img, band).values / 10000) 
            
    img = np.concatenate(band_list, axis = 0)
    img = np.transpose(img, [1,2,0])
    img = np.fliplr(img).copy()
    img = np.flipud(img).copy()

    H, W = img.shape[:2]
        
    # padding
    if H != 21 and W != 21:
        zeros = np.zeros((1, 20, 12))
        img = np.concatenate((img, zeros), axis = 0)
        zeros = np.zeros((21, 1, 12))
        img = np.concatenate((img, zeros[:]), axis = 1)
        
    elif H != 21:
        zeros = np.zeros((1, 21, 12))
        img = np.concatenate((img, zeros), axis = 0)
        
    elif W != 21:
        zeros = np.zeros((21, 1, 12))
        img = np.concatenate((img, zeros[:]), axis = 1)
        

    return img

In [5]:
# incase we want to use a Dataloader, we could use this
class CustomImageDataset(Dataset):
    def __init__(self, label_dir, img_dir, transform=None, target_transform=None):
        self.img_labels = list(np.load(label_dir))
        self.img_dir = img_dir
        image_paths = list(np.load(img_dir))
        self.image_paths = [path[1:] for path in image_paths]
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        image = load_image(self.image_paths[idx])
        
        # convert to float32
        image = np.float32(image)
        label = self.img_labels[idx]
        #image = image[:, :, [0,1,5,9,10,11]]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [6]:
from torch.utils.data import WeightedRandomSampler

BATCH_SIZE = 10
SHUFFLE = False

# Train augmentations
transformation = v2.Compose([
    v2.RandomHorizontalFlip(p=0.5),
    v2.RandomVerticalFlip(p=0.5),
    #v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=utils.MEANS, std=utils.STDS),
    transforms.ToTensor()
])
# Validation augmentation
transformation_val = v2.Compose([
    v2.Normalize(mean=utils.MEANS, std=utils.STDS),
    transforms.ToTensor()
])

train_data = CustomImageDataset(os.path.join(BASE_PATH_DATA, "skogs_gts_train.npy"), os.path.join(BASE_PATH_DATA, 'skogs_names_train.npy'), transform=transformation)
labels = train_data.img_labels
class_sample_count = np.array(
    [len(np.where(labels == t)[0]) for t in np.unique(labels)])

weight = 1. / class_sample_count
samples_weight = np.array([weight[t] for t in labels])

samples_weight = torch.from_numpy(samples_weight)
sampler = WeightedRandomSampler(samples_weight.type('torch.DoubleTensor'), len(samples_weight))

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=SHUFFLE, sampler=sampler)

val_data = CustomImageDataset(os.path.join(BASE_PATH_DATA, "skogs_gts_val.npy"), os.path.join(BASE_PATH_DATA, 'skogs_names_val.npy'), transform=transformation_val)
val_loader = DataLoader(val_data, batch_size=len(val_data), shuffle=SHUFFLE)

test_data = CustomImageDataset(os.path.join(BASE_PATH_DATA, "skogs_gts_test.npy"), os.path.join(BASE_PATH_DATA, 'skogs_names_test.npy'), transform=transformation_val)
test_loader = DataLoader(test_data, batch_size=len(test_data), shuffle=SHUFFLE)

In [7]:
# train model function
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs = 10, show_plot = True):
    
    min_loss = 10000
    max_acc = 0
    # Track loss
    training_loss, validation_loss = [], []
    
    # Track accuracy
    training_acc, validation_acc = [], []
    
    for i in range(num_epochs):
        # Track loss
        epoch_training_loss, epoch_validation_loss = 0, 0
        train_size, val_size = 0, 0
        
        # track accuracy
        train_correct, val_correct = 0, 0

        # training
        model.train(True)
        for batch_nr, (data, labels) in enumerate(train_loader):
            # predict
            data = data.to(device)
            labels = labels.to(device)
            
            pred = model(data)
            pred = torch.squeeze(pred)

            # calculate accuracy
            preds = torch.sigmoid(pred) >= 0.5
            
            train_correct += torch.sum(preds==labels).item()
            
            # Clear stored gradient values
            optimizer.zero_grad()
            loss = criterion(pred, labels.float())
            
            # Backpropagate the loss through the network to find the gradients of all parameters
            loss.backward()
            
            # Update the parameters along their gradients
            optimizer.step()
            
            # Update loss
            epoch_training_loss += loss.cpu().detach().numpy()
            train_size = batch_nr
            
        # validation
        model.eval()
        for data, labels in val_loader:
            data = data.to(device)
            labels = labels.to(device)
            # predict

            pred = model(data)
            pred = torch.squeeze(pred)

            # calculate accuracy
            preds = torch.sigmoid(pred) >= 0.5
            
            val_correct += torch.sum(preds==labels).item()
             
            # calculate loss
            loss = criterion(pred, labels.float())
            
            # check if loss is smaller than before, if so safe model
            if loss<min_loss:
                torch.save(model, 'best_model_sig-gpu.pt')
                min_loss = loss

            if max_acc<(val_correct/len(val_data)):
                torch.save(model, 'best_model_acc.pt')
                max_acc = val_correct/len(val_data)
            
            # Save loss for plot
            validation_loss.append(loss.cpu().detach().numpy())
            
            
        # Save loss for plot
        training_loss.append(epoch_training_loss/train_size)
        
        
        # Save accuracy for plot
        training_acc.append(train_correct/len(train_data))
        validation_acc.append(val_correct/len(val_data))

        # Print loss every 5 epochs
        #if i % 5 == 0:
        print(f'Epoch {i}, training loss: {training_loss[-1]}, validation loss: {validation_loss[-1]}')
        print(f'Train accuracy = {train_correct/len(train_data)}')
        print(f'Validation accuracy = {val_correct/len(val_data)}')

        torch.save(model, 'latest_model_sig-gpu.pt')
        
    if show_plot:
        # Plot training and validation loss
        epoch = np.arange(len(training_loss))
        plt.figure(figsize=(8,4), dpi=100)
        plt.plot(epoch, training_loss, 'r', label='Training loss',)
        plt.plot(epoch, validation_loss, 'b', label='Validation loss')
        plt.legend()
        plt.xlabel('Epoch'), plt.ylabel('Loss')
        plt.show()
        
        # Plot training and validation accuracy
        plt.figure(figsize=(8,4), dpi=100)
        plt.plot(epoch, training_acc, 'r', label='Training accuracy',)
        plt.plot(epoch, validation_acc, 'b', label='Validation accuracy')
        plt.legend()
        plt.xlabel('Epoch'), plt.ylabel('Accuracy')
        plt.show()

In [ ]:
# Hyperparams
LEARNING_RATE = 0.01
EPOCHS = 100

# define network
network = models.resnet18(num_classes = 1)
network.conv1 = nn.Conv2d(12, 64, kernel_size=3, stride=1, padding=3) # 12 bands = 12 input channels

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
network.to(device)

# define loss
loss_function = torch.nn.BCEWithLogitsLoss().to(device)
# define optimizer
optimizer = torch.optim.Adam(network.parameters(), lr=LEARNING_RATE)

train_model(network, loss_function, optimizer, train_loader, val_loader, EPOCHS)

Epoch 0, training loss: 0.9870688343048095, validation loss: 0.6440809965133667
Train accuracy = 0.5653846153846154
Validation accuracy = 0.6944444444444444
Epoch 1, training loss: 0.5372790515422821, validation loss: 0.5908358693122864
Train accuracy = 0.7153846153846154
Validation accuracy = 0.8055555555555556
Epoch 2, training loss: 0.4986774015426636, validation loss: 0.5724557042121887
Train accuracy = 0.75
Validation accuracy = 0.6805555555555556
Epoch 3, training loss: 0.4521308797597885, validation loss: 0.48530319333076477
Train accuracy = 0.8192307692307692
Validation accuracy = 0.75
Epoch 4, training loss: 0.4800739407539368, validation loss: 0.4048489034175873
Train accuracy = 0.7807692307692308
Validation accuracy = 0.8472222222222222
Epoch 5, training loss: 0.48147916853427886, validation loss: 0.5996066331863403
Train accuracy = 0.7769230769230769
Validation accuracy = 0.75
Epoch 6, training loss: 0.49827021449804304, validation loss: 0.45844125747680664
Train accuracy =

In [ ]:
# Testing
correct = 0
predictions = []
cloudy_pred = 0
clear_pred = 0

model = torch.load('best_model_sig-gpu.pt')

with torch.no_grad():
    for batch_nr, (data, labels) in enumerate(test_loader):
            data = data.to(device)
            labels = labels.to(device)
            # predict
            pred = model(data)
            pred = torch.squeeze(pred)
        
            # calculate accuracy
            preds = torch.sigmoid(pred) >= 0.5
            
            for pred in preds:
                predictions.append(pred.cpu())

for idx, pred in enumerate(predictions):
    if pred == test_label[idx]:
        correct += 1
        if pred == 1:
            cloudy_pred += 1
        else:
            clear_pred += 1

print("Final accuracy: %.2f%%" % (100*correct/len(test_label)))
print(f'Correct {correct} times out of {len(img_paths_test)}')


total_cloudy = np.sum(test_label)
total_clear = len(test_label)-total_cloudy

print(f'Correct {clear_pred} times out of {total_clear}: {100*clear_pred/total_clear}')
print(f'Correct {cloudy_pred} times out of {total_cloudy}: {100*cloudy_pred/total_cloudy}')

cm = confusion_matrix(test_label, predictions)
ConfusionMatrixDisplay(confusion_matrix = cm,  display_labels=['Clear', 'Cloudy']).plot()

In [ ]:
recall_clear = clear_pred/total_clear
recall_cloudy = cloudy_pred/total_cloudy
precision_clear = clear_pred/(clear_pred+(total_cloudy-cloudy_pred))
precision_cloudy = cloudy_pred/(cloudy_pred+(total_clear-clear_pred))
f1_clear = (2*precision_clear*recall_clear)/(precision_clear+recall_clear)
f1_cloudy = (2*precision_cloudy*recall_cloudy)/(precision_cloudy+recall_cloudy)

f1_avg = (f1_clear+f1_cloudy)/2
rec_avg = (recall_clear+recall_cloudy)/2
prec_avg = (precision_clear+precision_cloudy)/2

print(f'f1 avg {f1_avg:.4f}')

print(f'f1 clear {f1_clear:.4f}')

print(f'f1 cloudy {f1_cloudy:.4f}')

print()

print(f'recall avg {rec_avg:.4f}')

print(f'recall clear {recall_clear:.4f}')

print(f'recall cloudy {recall_cloudy:.4f}')

print()

print(f'precision avg {prec_avg:.4f}')

print(f'precision clear {precision_clear:.4f}')

print(f'precision cloudy {precision_cloudy:.4f}')



|            | Article       | ResNet18       |
| ----------- | ----------- | ----------- |
| **f1-avg** | 0.90 | **0.95**|
| **f1-clear** | 0.94 | **0.97**|
| **f1-cloudy** | 0.86 |**0.93**|
| **rec-avg** | 0.94 |**0.94** |
| **rec-clear** | 0.91 |**0.99** |
| **rec-cloudy** | 0.88 | **0.89**|
| **prec-avg** | 0.91 |**0.96** |
| **prec-clear** | 0.95 |**0.96** |
| **prec-cloudy** | 0.85 |**0.96**|



|            | MLP-5       | MLP-5-ens-10       | ResNet18-cls    |     | MLP       |CNN       |ResNet18       |
| ----------- | ----------- | ----------- |----------- |----------- |----------- |----------- |----------- |
| **f1-avg** | 0.73 | 0.88|0.90| | 0.89 | 0.83|**0.95**|
| **f1-clear** | 0.81 | 0.94|0.94|| 0.94 | 0.91|**0.97**|
| **f1-cloudy** | 0.65 |0.82|0.86|| 0.84 | 0.75|**0.93**|
| **rec-avg** | 0.73 |0.86 |0.91||0.89 | 0.82|**0.94**|
| **rec-clear** | 0.77 |0.97 |0.94||0.93 | 0.93|**0.99**|
| **rec-cloudy** | 0.68 | 0.75|0.88||0.85 | 0.71|**0.89**|
| **prec-avg** | 0.74 |0.91 |0.90||0.88 | 0.85|**0.96**|
| **prec-clear** | 0.85 |0.91 |0.95||0.94 | 0.89|**0.96**|
| **prec-cloudy** | 0.63 |0.91|0.85||0.82 | 0.80|**0.96**|

